## Environment Setting
Google drive mount (for Colab users) and package importing.
You can optionally install and import torchensemble package for ensemble learning

In [ ]:
# For Colab users
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.insert(0,'/content/drive/{path to project directory}')

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import random  #
import argparse  #
import torchvision.transforms as transforms  #
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from data_utils import Mydataset, Mytensordataset, collate_fn
from models import ConvLSTM

In [ ]:
!pip install git+https://github.com/snuml2021tmp/Ensemble-Pytorch.git

In [ ]:
# Only For Ensemble
from torchensemble import GradientBoostingClassifier
from torchensemble import SoftGradientBoostingClassifier
from torchensemble import VotingClassifier
from torchensemble import BaggingClassifier
from torchensemble import FusionClassifier
from torchensemble import SnapshotEnsembleClassifier
from torchensemble.utils.logging import set_logger
from torchensemble.utils import io

## (Optional) Sample Visualization
You can see actual sample images and sorted class indices. Additional matplotlib package is needed.

In [ ]:
# Just for reference: see actual samples
import matplotlib.pyplot as plt

alphabet = {
        'A(a)' : '0', 
        'B(b)' : '1', 
        'C(c)' : '2', 
        'D(d)' : '3', 
        'E(e)' : '4', 
        'F(f)' : '5', 
        'G(g)' : '6', 
        'H(h)' : '7', 
        'I(i)' : '8', 
        'J(j)' : '9', 
        'K(k)' : '10', 
        'L(l)' : '11', 
        'M(m)' : '12', 
        'N(n)' : '13', 
        'O(o)' : '14', 
        'P(p)' : '15', 
        'Q(q)' : '16', 
        'R(r)' : '17', 
        'S(s)' : '18', 
        'T(t)' : '19', 
        'U(u)' : '20', 
        'V(v)' : '21', 
        'W(w)' : '22', 
        'X(x)' : '23', 
        'Y(y)' : '24', 
        'Z(z)' : '25'
    }

In [ ]:
# Just for reference: see actual samples
idx = 10
sample = np.load(f'./data/emnist/train/numpy/{idx}.npy')
sample_target = np.loadtxt('./data/emnist/train/label.txt')[idx]

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(1, 10, i+1)
    ax = plt.gca()
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    plt.imshow(sample[i], cmap='gray')
    
plt.show()
print("sorted label: ", end=' ')
label_str = '('
for i in range(10):
    print(int(sample_target[i].item()), end=' ')
    label_str += " " + list(alphabet.keys())[int(sample_target[i].item())]
label_str += " )"
print()
print(label_str)

In [ ]:
# Use 0th GPU for training
torch.cuda.set_device(0)

In [ ]:
# fix random seed to increase reproducibility
# NOTE: Do not modify here!
SEQUENCE_LENGTH = 10
NUM_CLASSES = 26

random_seed = 7
torch.manual_seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
%env CUBLAS_WORKSPACE_CONFIG=:16:8

def seed_worker(worker_seed):
    np.random.seed(worker_seed)
    random.seed(worker_seed)

num_workers = 8

In [ ]:
# NOTE: you can modify mean and std for normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [ ]:
print_interval = 15
max_epoch = 1
batch_size = 256

In [ ]:
# NOTE: modify path for your setting

# Option 1: use Mydataset (both for local and Colab users)
train_ds = Mydataset('./data/emnist/train/numpy', './data/emnist/train/label.txt', transform=transform)
valid_ds = Mydataset('./data/emnist/valid/numpy', './data/emnist/valid/label.txt', False, transform=transform)

# Option 2: use Mytensordataset (Colab users should use this)
# train_ds = Mytensordataset('./data/emnist/Colab/train/img.pt', './data/emnist/Colab/train/label.pt', True, transform=transform)
# valid_ds = Mytensordataset('./data/emnist/Colab/valid/img.pt', './data/emnist/Colab/valid/label.pt', False, transform=transform)

train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, worker_init_fn=seed_worker, collate_fn=collate_fn, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, num_workers=num_workers, worker_init_fn=seed_worker, collate_fn=collate_fn, shuffle=True)

In [ ]:
# You can add or modify your ConvLSTM's hyperparameter (keys and values)
kwargs = {
    'cnn_input_dim': 1,
    'rnn_hidden_size': 8,
    'rnn_num_layers': 1,
    'rnn_dropout': 0.1
}

In [ ]:
# NOTE: you can freely modify or add training hyperparameters
print_interval = 15
max_epoch = 15

In [ ]:
# Non-ensemble learning
model = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kargs).cuda()
print(model)
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
model_optim = 
loss_func =
# NOTE: you can define additional components
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
def train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, load_path=None, save_path='./model.pt'):
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    # Load your states
    loaded_epoch = 0
    loaded_best_acc = -1
    if load_path is not None:
        state = torch.load(load_path)
        model.load_state_dict(state["model"])
        model_optim.load_state_dict(state["optimizer"])
        loaded_epoch = state["epoch"]
        loaded_best_acc = state["best_acc"]
        # ...
        
    ##############################################################################
    #                          END OF YOUR CODE                                  #
    ##############################################################################
    
    best_valid_accuracy = 0 if loaded_best_acc == -1 else loaded_best_acc

    for epoch in np.array(list(range(max_epoch - loaded_epoch))) + loaded_epoch:
        n_samples = 0
        n_correct = 0
        model.train()
        for step, sample in enumerate(train_dl):
            img, label = sample  # (BxT, C=1, H, W), (BxT)
            img = img.cuda()
            label = label.cuda()
            outputs = model((img, label))
            ##############################################################################
            #                          IMPLEMENT YOUR CODE                               #
            ##############################################################################
            # Problem5: implement optimization part (about four short lines are sufficient)

            ##############################################################################
            #                          END OF YOUR CODE                                  #
            ##############################################################################
            n_samples += img.size(0)
            n_correct += (outputs.argmax(-1) == label).sum().item()
            if (step + 1) % print_interval == 0:
                print('epoch:', epoch + 1, 'step:', step + 1, 'loss:', loss.item(), 'accuracy:', 100 * (n_correct / n_samples))
                
        n_samples = 0
        n_correct = 0
        with torch.no_grad():
            model.eval()
            for step, sample in enumerate(valid_dl):
                img, label = sample
                img = img.cuda()
                outputs = model(img)
                pred = np.argmax(outputs.cpu().data.numpy(), axis=1)
                label = label.data.numpy()
                n_samples += label.shape[0]
                n_correct += (pred == label).astype(float).sum()
            valid_accuracy = 100 * (n_correct/n_samples)
            if valid_accuracy > best_valid_accuracy:
                print("New best valid accuracy, saving model")
                ##############################################################################
                #                          IMPLEMENT YOUR CODE                               #
                ##############################################################################
                # Save your states
                state = {
                    "model": model.state_dict(),
                    "optimizer": model_optim.state_dict(),
                    "epoch": epoch + 1,
                    "best_acc": best_valid_accuracy,
                    # ...
                }
                ##############################################################################
                #                          END OF YOUR CODE                                  #
                ##############################################################################
                torch.save(state, save_path)
                best_valid_accuracy = valid_accuracy
            print('Valid epoch: %d, Valid accuracy: %.2f, Best valid accuracy: %.2f' % (epoch + 1, valid_accuracy, best_valid_accuracy))

In [ ]:
def eval(valid_dl, load_path):
    state = torch.load(load_path)
    model.load_state_dict(state["model"])
    n_samples = 0
    n_correct = 0
    with torch.no_grad():
        model.eval()
        for step, sample in enumerate(valid_dl):
            img, label = sample
            img = img.cuda()
            outputs = model(img)
            pred = np.argmax(outputs.cpu().data.numpy(), axis=1)
            label = label.data.numpy()
            n_samples += label.shape[0]
            n_correct += (pred == label).astype(float).sum()
        valid_accuracy = 100 * (n_correct/n_samples)
        print('Valid accuracy: %.2f' % (valid_accuracy))

In [ ]:
load_path = None
train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, load_path=load_path, save_path='./model.pt')

In [ ]:
# load and evaluate non-ensemble model
load_path = './model.pt'
eval(valid_dl, load_path)

In [ ]:
####### Optional: ENSEMBLE - model type / optimizer / scheduler ########
estimator = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
# set ensemble model, optimizer, (optionally) lr scheduler
model = 
model.set_optimizer(
    
)
# Note: learning rate scheduler is optional
model.set_scheduler(

)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
logger = set_logger("Start Logging", use_tb_logger=False)
# epoch, optimizers, scheduler, best_acc, est_idx = io.load(model, './', use_scheduler=True)  # For load-and-rerun
model.fit(
    train_dl,
    epochs=max_epoch,
    log_interval=print_interval,
    test_loader=valid_dl,
    save_dir='./',
    # retrain=True,  # For load-and-rerun
    # loaded_optimizers=optimizers,  # For load-and-rerun
    # loaded_scheduler=scheduler,  # For load-and-rerun
    # loaded_epoch=epoch,  # For load-and-rerun
    # loaded_est_idx=est_idx,  # For load-and-rerun
    # loaded_best_acc=best_acc,  # For load-and-rerun
)
acc = model.evaluate(valid_dl)
print("Valid accuracy: %.2f" % acc)

In [ ]:
# load and evaluate historical best model ensemble model
estimator = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
# Note: set ensemble type, optimizer, and scheduler exactly same with saved model
model = 
model.set_optimizer(
    
)
model.set_scheduler(

)
_, _, _, _ = io.load(model, './')

In [ ]:
model.evaluate(valid_dl)